# 🧪 QEPC Sandbox v2

A safe playground to experiment with the QEPC NBA engine:

1. Environment & imports  
2. (Optional) System diagnostics  
3. Load schedule & select games  
4. Build team strengths  
5. (Optional) Apply injury overrides  
6. Compute λ (expected points)  
7. Run Poisson simulations  
8. View and explore results  
9. (Optional) Interactive widgets


## 1. Environment & Project Setup


In [2]:
import sys
from pathlib import Path

# Try to detect the QEPC project root
cwd = Path.cwd()
candidate_roots = [cwd, cwd.parent, cwd.parent.parent]

project_root = None
for cand in candidate_roots:
    if (cand / "qepc").is_dir() and (cand / "qepc_autoload.py").exists():
        project_root = cand
        break

if project_root is None:
    project_root = cwd

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("📁 QEPC project root set to:", project_root)

try:
    from notebook_context import *
    print("✅ notebook_context imported.")
except ImportError:
    print("ℹ️ notebook_context not found; continuing without it.")

try:
    import qepc_autoload as qa
    print("✅ qepc_autoload imported as qa.")
except Exception as e:
    print("❌ Error importing qepc_autoload:", e)


📁 QEPC project root set to: C:\Users\wdors\qepc_project
[QEPC Paths] Project Root set: C:\Users\wdors\qepc_project
[QEPC] Autoload complete.
[QEPC] Root Shim Restored. Forwarding to qepc.autoload...
✅ notebook_context imported.
✅ qepc_autoload imported as qa.


## 2. System Diagnostics (Optional)


In [3]:
# === QEPC Sandbox: System Diagnostics (Optional) ===

from qepc.utils.diagnostics import run_system_check

diagnostic_report = run_system_check()
diagnostic_report  # Shows root/files/modules info as a dict


🚀 QEPC SYSTEM DIAGNOSTICS INITIALIZED...

✅ Project Root: OK – Resolved to C:\Users\wdors\qepc_project

🔍 Checking required files...
✅ Canonical Schedule: OK – C:\Users\wdors\qepc_project\data\Games.csv
✅ Raw Player Stats: OK – C:\Users\wdors\qepc_project\data\raw\PlayerStatistics.csv
✅ Raw Team Stats: OK – C:\Users\wdors\qepc_project\data\raw\TeamStatistics.csv
✅ Autoload Context: OK – C:\Users\wdors\qepc_project\qepc_autoload.py
✅ Restore Guide (Notebook): OK – C:\Users\wdors\qepc_project\RESTORE_GUIDE.ipynb
✅ Restore Guide (Markdown): OK – C:\Users\wdors\qepc_project\notebooks\RESTORE_GUIDE.md


Check,Status,Details
Canonical Schedule,OK,C:\Users\wdors\qepc_project\data\Games.csv
Raw Player Stats,OK,C:\Users\wdors\qepc_project\data\raw\PlayerStatistics.csv
Raw Team Stats,OK,C:\Users\wdors\qepc_project\data\raw\TeamStatistics.csv
Autoload Context,OK,C:\Users\wdors\qepc_project\qepc_autoload.py
Restore Guide (Notebook),OK,C:\Users\wdors\qepc_project\RESTORE_GUIDE.ipynb
Restore Guide (Markdown),OK,C:\Users\wdors\qepc_project\notebooks\RESTORE_GUIDE.md



📊 Checking data schemas (where files exist)...
✅ Schema: data/Games.csv: OK – All expected columns present.
✅ Schema: data/Team_Stats.csv: OK – All expected columns present.
✅ Schema: data/raw/PlayerStatistics.csv: OK – All expected columns present.
✅ Schema: data/raw/TeamStatistics.csv: OK – All expected columns present.


Check,Status,Details
data/Games.csv,OK,All expected columns present.
data/Team_Stats.csv,OK,All expected columns present.
data/raw/PlayerStatistics.csv,OK,All expected columns present.
data/raw/TeamStatistics.csv,OK,All expected columns present.



🧪 Checking key module imports...
✅ Module: qepc.autoload.paths: OK – Loaded
✅ Module: qepc.core.lambda_engine: OK – Loaded
✅ Module: qepc.core.simulator: OK – Loaded
✅ Module: qepc.sports.nba.sim: OK – Loaded
✅ Module: qepc.sports.nba.strengths_v2: OK – Loaded
✅ Module: qepc.sports.nba.player_data: OK – Loaded
✅ Module: qepc.sports.nba.opponent_data: OK – Loaded
✅ Module: qepc.utils.backup: OK – Loaded
✅ Module: qepc.backtest.backtest_engine: OK – Loaded


Check,Status,Details
qepc.autoload.paths,OK,Loaded successfully
qepc.core.lambda_engine,OK,Loaded successfully
qepc.core.simulator,OK,Loaded successfully
qepc.sports.nba.sim,OK,Loaded successfully
qepc.sports.nba.strengths_v2,OK,Loaded successfully
qepc.sports.nba.player_data,OK,Loaded successfully
qepc.sports.nba.opponent_data,OK,Loaded successfully
qepc.utils.backup,OK,Loaded successfully
qepc.backtest.backtest_engine,OK,Loaded successfully



✨ DIAGNOSTICS COMPLETE.


{'project_root': 'C:\\Users\\wdors\\qepc_project',
 'files': [('Canonical Schedule',
   'OK',
   'C:\\Users\\wdors\\qepc_project\\data\\Games.csv'),
  ('Raw Player Stats',
   'OK',
   'C:\\Users\\wdors\\qepc_project\\data\\raw\\PlayerStatistics.csv'),
  ('Raw Team Stats',
   'OK',
   'C:\\Users\\wdors\\qepc_project\\data\\raw\\TeamStatistics.csv'),
  ('Autoload Context',
   'OK',
   'C:\\Users\\wdors\\qepc_project\\qepc_autoload.py'),
  ('Restore Guide (Notebook)',
   'OK',
   'C:\\Users\\wdors\\qepc_project\\RESTORE_GUIDE.ipynb'),
  ('Restore Guide (Markdown)',
   'OK',
   'C:\\Users\\wdors\\qepc_project\\notebooks\\RESTORE_GUIDE.md')],
 'schemas': [('data/Games.csv', 'OK', 'All expected columns present.'),
  ('data/Team_Stats.csv', 'OK', 'All expected columns present.'),
  ('data/raw/PlayerStatistics.csv', 'OK', 'All expected columns present.'),
  ('data/raw/TeamStatistics.csv', 'OK', 'All expected columns present.')],
 'modules': [('qepc.autoload.paths', 'OK', 'Loaded successfully')

## 3. Load NBA Schedule


In [4]:
# === QEPC Sandbox: Load NBA Schedule ===

import qepc_autoload as qa

schedule = qa.load_nba_schedule()
print("Number of games in schedule:", len(schedule))
schedule.head()


[QEPC NBA Sim] Successfully loaded and parsed 771 games from original format.
Number of games in schedule: 771


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00
4,10/22/2025,7:00 PM,Miami Heat,Orlando Magic,Kia Center,Regular Season,2025-10-22 19:00:00


## 4. Select Games to Model


In [5]:
# === QEPC Sandbox: Select Games to Model ===

# Option A: first 4 games (e.g., opening night)
games_to_model = schedule.head(4).copy()

print("Using these games:")
games_to_model


Using these games:


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00


## 5. Build Advanced Team Strengths


In [6]:
# === QEPC Sandbox: Build Advanced Team Strengths ===

from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

advanced_strengths = calculate_advanced_strengths()
print("Raw advanced_strengths shape:", advanced_strengths.shape)

# Collapse to one row per team
advanced_team_strengths = (
    advanced_strengths
    .groupby("Team", as_index=False)
    .mean(numeric_only=True)
)

print("Unique teams in advanced strengths:", len(advanced_team_strengths))
advanced_team_strengths.head()


[QEPC Strength V2] Starting Advanced Calculation (Cutoff: Now)...
[QEPC PlayerData] Successfully loaded 1635462 rows from PlayerStatistics.csv.
[QEPC Opponent Processor] Loading raw team data for Weighted DRtg...
[QEPC Opponent Processor] Calculated Weighted DRtg for 30 teams.
[QEPC Strength V2] Calculated Time-Travel Strengths for 30 teams.
Raw advanced_strengths shape: (30, 5)
Unique teams in advanced strengths: 30


,Team,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725
1,Boston Celtics,122.0,107.711196,68.080000,12.410859
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424


## 6. Injury Overrides (Optional)


In [7]:
# === QEPC Sandbox: Injury Overrides (Optional) ===

import pandas as pd

inj_path = project_root / "data" / "Injury_Overrides.csv"

if inj_path.exists():
    injuries = pd.read_csv(inj_path)
    print("✅ Loaded injury overrides from:", inj_path)
    display(injuries)

    # Collapse to team-level impact (multiply multiple entries per team)
    team_injury_impact = (
        injuries
        .groupby("Team", as_index=False)["Impact"]
        .prod()
        .rename(columns={"Impact": "InjuryImpact"})
    )

    print("Team-level InjuryImpact:")
    display(team_injury_impact)

    # Merge into strengths
    inj_adjusted = advanced_team_strengths.merge(
        team_injury_impact,
        on="Team",
        how="left"
    )

    # Teams without overrides get impact = 1.0 (no change)
    inj_adjusted["InjuryImpact"] = inj_adjusted["InjuryImpact"].fillna(1.0)

    # Apply to ORtg (you can also apply to Pace if you want)
    inj_adjusted["ORtg_inj"] = inj_adjusted["ORtg"] * inj_adjusted["InjuryImpact"]

    print("Injury-adjusted team strengths (first 5):")
    display(inj_adjusted.head())

    # This is the table we feed into the λ engine
    team_strengths_for_lambda = inj_adjusted.copy()
    team_strengths_for_lambda["ORtg"] = team_strengths_for_lambda["ORtg_inj"]

    # Optional clean-up of helper columns
    for col in ["ORtg_inj", "InjuryImpact"]:
        if col in team_strengths_for_lambda.columns:
            team_strengths_for_lambda.drop(columns=[col], inplace=True)

else:
    print("ℹ️ No Injury_Overrides.csv found at", inj_path)
    print("Using unadjusted advanced strengths.")
    team_strengths_for_lambda = advanced_team_strengths.copy()


✅ Loaded injury overrides from: C:\Users\wdors\qepc_project\data\Injury_Overrides.csv


,Team,PlayerName,Status,Impact,Note
0,Indiana Pacers,Tyrese Haliburton,out,0.85,Out for season


Team-level InjuryImpact:


,Team,InjuryImpact
0,Indiana Pacers,0.85


Injury-adjusted team strengths (first 5):


,Team,ORtg,DRtg,Pace,Volatility,InjuryImpact,ORtg_inj
0,Atlanta Hawks,122.0,109.681259,71.940000,10.262725,1.0,122.0
1,Boston Celtics,122.0,107.711196,68.080000,12.410859,1.0,122.0
2,Brooklyn Nets,122.0,118.555811,73.044706,9.097107,1.0,122.0
3,Charlotte Hornets,122.0,116.165573,70.528421,13.022697,1.0,122.0
4,Chicago Bulls,122.0,113.762988,69.166667,9.479424,1.0,122.0


## 7. Compute Lambda (Expected Points)


In [8]:
# === QEPC Sandbox: Compute Lambda (Expected Points) ===

from qepc.core.lambda_engine import compute_lambda

lambda_df = compute_lambda(games_to_model, team_strengths_for_lambda)

print("Lambda dataframe columns:")
print(lambda_df.columns.tolist())

# Adjust these column names if needed based on the printout above
display(
    lambda_df[
        ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
    ]
)


[QEPC Lambda] Computed lambda & volatility for 4 games.
Lambda dataframe columns:
['Date', 'Time', 'Away Team', 'Home Team', 'Venue', 'Notes', 'gameDate', 'lambda_home', 'lambda_away', 'vol_home', 'vol_away']


,Away Team,Home Team,lambda_away,lambda_home,vol_away,vol_home
0,Houston Rockets,Oklahoma City Thunder,87.431249,96.878531,32.325436,14.033294
1,Golden State Warriors,Los Angeles Lakers,100.679142,101.130359,10.256705,13.236314
2,Brooklyn Nets,Charlotte Hornets,107.385905,112.664166,9.097107,13.022697
3,Cleveland Cavaliers,New York Knicks,103.931072,107.670313,12.175616,13.069948


## 8. Run Simulation & View Results


In [9]:
# === QEPC Sandbox: Run QEPC Simulation ===

from qepc.core.simulator import run_qepc_simulation

sim_results = run_qepc_simulation(lambda_df, num_trials=20000)

print("Simulation result columns:")
print(sim_results.columns.tolist())
sim_results.head()


[QEPC Simulator] Running 20000 trials for 4 games (Chaos Engine Active)...
[QEPC Simulator] Simulation complete.
Simulation result columns:
['Date', 'Time', 'Away Team', 'Home Team', 'Venue', 'Notes', 'gameDate', 'lambda_home', 'lambda_away', 'vol_home', 'vol_away', 'Home_Win_Prob', 'Away_Win_Prob', 'Tie_Prob', 'Expected_Score_Total', 'Expected_Spread', 'Sim_Home_Score', 'Sim_Away_Score']


,Date,Time,Away Team,Home Team,Venue,Notes,gameDate,lambda_home,lambda_away,vol_home,vol_away,Home_Win_Prob,Away_Win_Prob,Tie_Prob,Expected_Score_Total,Expected_Spread,Sim_Home_Score,Sim_Away_Score
0,10/21/2025,7:30 PM,Houston Rockets,Oklahoma City Thunder,Paycom Center,Regular Season,2025-10-21 19:30:00,96.878531,87.431249,14.033294,32.325436,0.65345,0.32980,0.01675,184.48425,9.08155,96.78290,87.70135
1,10/21/2025,10:00 PM,Golden State Warriors,Los Angeles Lakers,Crypto.com Arena,Regular Season,2025-10-21 22:00:00,101.130359,100.679142,13.236314,10.256705,0.49950,0.47855,0.02195,201.94200,0.47270,101.20735,100.73465
2,10/22/2025,7:00 PM,Brooklyn Nets,Charlotte Hornets,Spectrum Center,Regular Season,2025-10-22 19:00:00,112.664166,107.385905,13.022697,9.097107,0.60645,0.36920,0.02435,220.01695,5.21185,112.61440,107.40255
3,10/22/2025,7:00 PM,Cleveland Cavaliers,New York Knicks,Madison Square Garden,Regular Season,2025-10-22 19:00:00,107.670313,103.931072,13.069948,12.175616,0.57765,0.40000,0.02235,211.70490,3.89980,107.80235,103.90255


## 9. Interactive QEPC Controls (Widgets)


In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output

from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation


# --- Build dropdown options from the schedule ---

def format_game_option(row):
    """Turn a schedule row into a nice label for the dropdown."""
    return f"{row['Date']} {row['Time']} – {row['Away Team']} @ {row['Home Team']}"


game_options = []
for idx, row in schedule.iterrows():
    label = format_game_option(row)
    # Each option is (label shown to you, underlying value = index in schedule)
    game_options.append((label, idx))

game_dropdown = widgets.Dropdown(
    options=game_options,
    description='Game:',
    layout=widgets.Layout(width='90%')
)

# Keep the trials slider – this still feels good as a slider
num_trials_slider = widgets.IntSlider(
    value=10000,
    min=1000,
    max=50000,
    step=1000,
    description='Trials:',
    continuous_update=False
)

run_button = widgets.Button(
    description="Run QEPC Sim",
    button_style='success',
    tooltip='Run QEPC for the selected game'
)

output = widgets.Output()


def on_run_clicked(b):
    with output:
        clear_output()

        # 1) Get the selected game from the dropdown
        game_idx = game_dropdown.value
        game_row = schedule.loc[[game_idx]].copy()  # keep it as a DataFrame

        print("Running QEPC for:")
        display(game_row[["Date", "Time", "Away Team", "Home Team", "Venue", "Notes"]])

        # 2) Compute lambda for this single game
        lambda_df = compute_lambda(game_row, team_strengths_for_lambda)

        print("\nLambda (expected points) for this game:")
        display(
            lambda_df[
                ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
            ]
        )

        # 3) Run simulation
        trials = num_trials_slider.value
        print(f"\nRunning simulation with {trials} trials...\n")
        sim_results = run_qepc_simulation(lambda_df, num_trials=trials)

        # 4) Show a compact summary (adjust column names if your version differs)
        cols = [
            "Away Team", "Home Team",
            "Home_Win_Prob", "Away_Win_Prob",
            "Expected_Score_Total", "Expected_Spread",
            "Sim_Home_Score", "Sim_Away_Score"
        ]
        cols = [c for c in cols if c in sim_results.columns]

        print("QEPC summary for this matchup:")
        display(sim_results[cols])


run_button.on_click(on_run_clicked)

controls = widgets.VBox([
    widgets.HTML("<h3>🎛 QEPC Interactive Controls</h3>"),
    game_dropdown,
    num_trials_slider,
    run_button
])

display(widgets.VBox([controls, output]))
